# VWM project: analyse and parse the anime dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math

The dataset was downloaded from this website:
https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews?resource=download

In [2]:
df = pd.read_csv("animesWorking.csv")

In [3]:
df.info()
#df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    18336 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    18605 non-null  float64
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       18732 non-null  float64
 10  img_url     19131 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.8+ MB


In [4]:
print(df.columns)

Index(['uid', 'title', 'synopsis', 'genre', 'aired', 'episodes', 'members',
       'popularity', 'ranked', 'score', 'img_url', 'link'],
      dtype='object')


In [5]:
genre_column = df.genre
print(genre_column)

0        ['Comedy', 'Sports', 'Drama', 'School', 'Shoun...
1        ['Drama', 'Music', 'Romance', 'School', 'Shoun...
2        ['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...
3        ['Action', 'Military', 'Adventure', 'Comedy', ...
4         ['Action', 'Mystery', 'Supernatural', 'Vampire']
                               ...                        
19306           ['Sci-Fi', 'Adventure', 'Comedy', 'Magic']
19307    ['Adventure', 'Fantasy', 'Magic', 'Martial Art...
19308                ['Mystery', 'Horror', 'Supernatural']
19309    ['Action', 'Adventure', 'Comedy', 'Historical'...
19310    ['Military', 'Sci-Fi', 'Space', 'Drama', 'Mecha']
Name: genre, Length: 19311, dtype: object


In [6]:
tags = set()
for item in genre_column:
    res = set(eval(item))
    tags.update(res)
tags = list(tags)
tags.sort()
print("dataset contains following tags:")
print(tags)
print("# of distinct tags: ", len(tags))

dataset contains following tags:
['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri']
# of distinct tags:  43


In [7]:
output = pd.DataFrame(index=range(df.shape[0]),columns=range(44))
output.columns = ['Title'] + tags
print(output.columns)
for index, row in df.iterrows():
    output.iloc[index][0] = row['title']
    row_tags = set(eval(row['genre']))
    for t in row_tags:
        output.iloc[index][t] = 1
output = output.fillna(0)
print("0/1 df created")
    

Index(['Title', 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons',
       'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical',
       'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military',
       'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance',
       'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai',
       'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports',
       'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri'],
      dtype='object')
0/1 df created


In [8]:
attribute_totals = list()
output_normalised = pd.DataFrame(index=range(output.shape[0]),columns=range(output.shape[1]))
output_normalised.columns = output.columns
for index, row in output.iterrows():
    cnt = 0
    for i in range(1,44):
        cnt += int(row[i])
    attribute_totals.append(cnt)
#print(len(attribute_totals))
#print(attribute_totals)
for index, row in output.iterrows():
    output_normalised.iloc[index][0] = row[0]
    if attribute_totals[index] == 0:
        num = 0
    else:
        num = 1/math.sqrt(attribute_totals[index])
    for i in range(1,44):
        if row[i] == 0:
            output_normalised.iloc[index][i] = 0
        else:
            output_normalised.iloc[index][i] = num
print("Normalised")



Normalised


In [9]:
#output_normalised.to_csv('items.csv')


In [10]:
#check = pd.read_csv("items.csv")
#print(check)

In [12]:
final = pd.read_csv("items.csv")
nr_items = final.shape[0]
print("nr_items", nr_items)
idf = pd.DataFrame(index=range(1),columns=range(final.shape[1]))
idf.columns = ['Title'] + tags
idf['Title'][0] = 'idf'
for name in tags:
    summ = 0
    for el in final[name]:
        if el > 0:
            summ += 1
    idf[name][0] = math.log(nr_items/summ,10)
for index, row in idf.iterrows():
    print(row.tolist())
#idf.to_csv('idf.csv')
#check = pd.read_csv("idf.csv")
#print(check)

nr_items 19287
['idf', 0.6607763179677045, 0.7879202994635734, 2.1614130395140676, 0.4753011587671395, 1.6630506575148583, 1.5035093058286846, 0.8085779375355088, 1.3794688001132849, 0.7456867971358444, 1.6886675848546933, 1.648776784127788, 0.874993716228969, 1.199617392184297, 1.5768437803464406, 2.2245668401275416, 0.8788948450118859, 1.2163370687990818, 1.6578988238884207, 1.2082603536878032, 1.4546760117960091, 0.9857573817806657, 1.3467449553046615, 1.4389275683513483, 1.80669818488731, 1.671422858605084, 0.9530282649897102, 1.9086877234246413, 1.004913987435148, 0.8359405873824265, 1.2931531926942037, 1.3887384629915982, 2.3457454278625347, 0.9194024650785984, 2.232186236997734, 0.9724528542690655, 1.5339855764978112, 1.3762438262699974, 1.430351658173298, 1.0515073175156429, 2.1005732496635545, 2.067780736267247, 2.393170077790673, 2.387637589190712]


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Matplotlib umožňuje změnit styl na jiný, než defaultní
matplotlib.style.use('ggplot')

# Bez násl. řádku někdy nefunguje vykreslování grafů v Jupyter noteboocích
%matplotlib inline 

### Graphs and tables for report

In [ ]:
print(output['Action'].sum())
print (tags)
zero = np.zeros(shape=len(tags))
print(zero)
serie = pd.Series(data=zero, index=tags)
for tag in tags:
    serie[tag] = output[tag].sum()
print(serie)

In [ ]:
serie.plot.bar(color='indigo', figsize=(10,10))
#plt.xticks(fontsize=5)
#plt.yticks(fontsize=5)
plt.savefig("figure.png")

In [ ]:
df = pd.read_csv("items.csv")

In [11]:
##Some additional testing

In [ ]:
print(df.columns)
for col in df.columns:
    df[col].isnull().sum()
print(df[df['Action'].isnull()])